In [ ]:
#sudo docker run --gpus all -p 8888:8888 -it --rm tensorflow/tensorflow:latest-gpu-jupyter
#jupyter notebook

#!pip install lambeq

In [ ]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
from utilities import *

In [2]:
#define atomic-types

N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

In [3]:
#parser declaration

bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [4]:
#ansatze declaration

tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})

In [ ]:
#data-extraction for classic pipeline (linux)

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/CPN_edited.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/ePurse_edited.csv", "n")

In [ ]:
#data-extraction for classic pipeline (win11)

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\CPN_edited.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\ePurse_edited.csv", "n")

In [6]:
save_data("train_data.txt", train_labels, train_circuits)
save_data("test_data.txt", test_labels, test_circuits)
save_data("eval_data.txt", eval_labels, eval_circuits)

#train_labels, train_circuits = load_data("train_data.txt")
#test_labels, test_circuits = load_data("test_data.txt")
#eval_labels, eval_circuits = load_data("eval_data.txt")

In [ ]:
#training block for classical pipeline

train_set, test_set = pip.create_dataset(train_labels[0:], train_circuits[0:]), pip.create_dataset(test_labels[0:], test_circuits[0:])
pip.create_trainer(train_circuits[0:], test_circuits[0:])
pip.train_model(train_set, test_set)
pip.plot() 

In [ ]:
model = PytorchModel.from_checkpoint(checkpoint_path = "/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/codice/runs/May09_15-04-56_archlinux/model.lt")
#model = PytorchModel.load(checkpoint_path = "/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/codice/runs/May09_15-04-56_archlinux/model.lt")
trainer = PytorchTrainer(
                model = model,
                loss_function = torch.nn.BCEWithLogitsLoss(),
                optimizer = torch.optim.AdamW,
                epochs = 70,
                evaluate_functions = eval_metrics,
                evaluate_on_train = True,
                learning_rate = LEARNING_RATE,
                verbose = "text",
                seed = CLASSIC_SEED
        )

train_set, test_set = pip.create_dataset(train_labels[128:], train_circuits[128:]), pip.create_dataset(test_labels[128:], test_circuits[128:])
trainer.fit(train_set, test_set, evaluation_step = 1, logging_step = 1)

In [ ]:
temp_dict = {}
for pos, diagram in enumerate(train_circuits):
    key = pos
    if temp_dict.get(key) == None:
        temp_dict[key] = len(diagram)
    else:
        temp_dict[key] += 1
        
train_dict = temp_dict
for key, val in train_dict.items():
    print(f"{key}   {val}")

In [8]:
temp_dict = {}
for diagram in test_circuits:
    key = len(diagram)
    if temp_dict.get(key) == None:
        temp_dict[key] = 1
    else:
        temp_dict[key] += 1
        
test_dict = dict(sorted(temp_dict.items()))

In [9]:
temp_dict = {}
for diagram in eval_circuits:
    key = len(diagram)
    if temp_dict.get(key) == None:
        temp_dict[key] = 1
    else:
        temp_dict[key] += 1
        
eval_dict = dict(sorted(temp_dict.items()))

In [22]:
train_tensors = []
test_tensors = []

for circuit in train_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    train_tensors.append(tensor)

for circuit in test_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    test_tensors.append(tensor)

In [ ]:
from lambeq import BobcatParser, AtomicType
from lambeq import TensorAnsatz
from discopy import Box, Ty, Dim, Word
# Define atomic types
N = AtomicType.NOUN
S = AtomicType.SENTENCE

sentence = 'John walks in the park'

# Get a string diagram
parser = BobcatParser(verbose='text')
tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2)})
diagram = parser.sentence2diagram(sentence)
tensor_diagram = tensor_ansatz(diagram)


pad_diagrams = [tensor_ansatz(parser.sentence2diagram("parola")) for i in range(1, 20)]

new_diagram = tensor_diagram
for pad in pad_diagrams:
    new_diagram = pad @ new_diagram

print(new_diagram)
print(len(new_diagram))
print(len(tensor_diagram))
